In [2]:
# -*-coding:utf-8-*-
import pandas as pd
import numpy as np
from  datetime  import  *  
import  time  
import jieba

In [4]:
data = pd.read_csv('data/news/user_click_data.txt',  sep='\t',
                   names=['user_no', 'news_no', 'read_time', 'title', 'content', 'release_time']) 
print len(data),
data.sort_values(by='read_time', inplace=True)
data = data[data['content'].notnull()] #新闻内容列为空则去掉
# print len(data)

In [3]:
print "总共新闻数:", len(set(data['news_no'].values))
data.head(10)

总共新闻数: 5866


,user_no,news_no,read_time,title,content,release_time,title_cut
0,36604,100479135,2014-03-01 00:00:20,“两高”明确行贿罪定罪量刑标准,受过 特别 国家 受贿罪 有期徒刑 经济损失 四种 主动 刑法 21 会令 破获 案件 ...,2012年12月31日18:52,行贿罪 标准 两高 定罪 量刑
1,36604,100247298,2014-03-01 00:02:04,百度推出招聘信息搜索平台,显示 二级 入口 姗姗 点击 挖 各大 汇总 高级 筹备 搬家 自有 zhaopin 王...,2011年04月12日14:18,推出 招聘 平台 信息 搜索 百度
2,36604,100112500,2014-03-01 00:02:41,进出口行和中信保注资或将减至500亿元,金融机构 显示 国家 需 金额 科学 体制 冲突 缓慢 甚 缺失 包括 完善 很难 矛盾...,2010年02月01日11:04,进出口 中信 亿元 行 保 注资 减至 500
3,36604,100479091,2014-03-01 00:03:27,对话反腐败研究专家李永忠,书记 政治局 事件 薄熙来 可谓 值得 20 委员 绝大多数 房叔 四句 新世纪 201...,2012年12月28日11:40,研究 反腐败 对话 专家 李永忠
4,36604,100490875,2014-03-01 00:04:23,盘点“到龄转任”省级党政高官,书记 青海 宁夏回族自治区政府 主任 中 常务 河南 山东 四川 民盟中央 刘怀廉 王秀...,2013年02月08日14:58,党政 省级 高官 盘点 到龄 转任
5,13544,100633200,2014-03-01 00:04:27,拥有私人飞机的中国富豪,巨人 型 G550 万美元 排 价值 商界 张 山水 CEO 6280 近东 苏宁 史玉柱 ...,NaN,中国富豪 飞机 私人 拥有
6,36604,100487408,2014-03-01 00:05:21,十八大后副大军区级将领调整基本到位,十一届 空军指挥学院 二炮 赵宗岐 河南 整体 殷方龙 中央政治局常委 外 俄罗斯 枣庄...,2013年01月29日15:44,十八 区级 大军 副 将领 大后 调整 到位
7,36604,100119052,2014-03-01 00:06:04,乔治·哈里森,修行 娴熟 与其说是 歌曲 内敛 衬托 萨 总统 淳朴 赢得 松散地 贾 榜首 远飞 世...,2002年01月05日00:00,哈里森 乔治
8,38007,100644648,2014-03-01 00:06:19,记者街头“摔倒”测试路人管不管,CFP3 尝试 27 日 测试 找到 月 下午 摔倒 分钟 救心丸 深圳华强 解思俊 北 速...,2014年02月28日10:30,记者 街头 摔倒 测试 路人管
9,930,100644648,2014-03-01 00:06:21,记者街头“摔倒”测试路人管不管,CFP3 尝试 27 日 测试 找到 月 下午 摔倒 分钟 救心丸 深圳华强 解思俊 北 速...,2014年02月28日10:30,记者 街头 摔倒 测试 路人管


In [4]:
import io
# 分词
def cut_messages(newslist, name=None, is_test_mode=False):
#     if (not is_test_mode) and is_exist_file(dir_path, name):
#         return load_from_pickle(dir_path, name)

    cut_news_list = []
    stop = [line.strip() for line in io.open('data/stop.txt', 'r', encoding='utf-8').readlines()]  # 停用词
    for news in newslist:
        fenci = jieba.cut(news, cut_all=False)  # False默认值：精准模式
        valid_words = list(set(fenci) - set(stop))
        cut_news_list.append(valid_words)
#     if not is_test_mode:
#         dump_to_pickle('data', name, cut_messages_list)
    cut_news_list = [" ".join(news) for news in cut_news_list]
    return cut_news_list

cut_news_list = cut_messages(data['content'])
data['content'] = cut_news_list

In [5]:
# data.to_csv('data/cut_user_click_data.txt', sep='\t', index=False, header=None, encoding='utf-8')

In [6]:
#处理时间
def process_date(dates, mode):
    def parse_date_timestamp(date):
        return datetime.fromtimestamp(date)
    
    def parse_date_str(date):
        format = "%Y-%m-%d %H:%M:%S"
        return datetime.strptime(date, format)
    
    return map(parse_date_timestamp, dates) if mode=='timestamp' else map(parse_date_str, dates)

read_time = process_date(data['read_time'], mode='timestamp')
data['read_time'] = read_time

In [7]:
def split_data(data):
    split_by_time = np.array(map(lambda date: True if date.day <= 20 else False, data['read_time']))
    train_data = data[split_by_time] # 94547
    test_data = data[~split_by_time] # 21678
    return train_data, test_data
    
train_data, test_data = split_data(data)

In [8]:
# encoding=utf-8
import logging
import numpy as np
from gensim import corpora
from gensim import models
from gensim import similarities
import pandas as pd
import matplotlib.pyplot as plt
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

texts = train_data['content'].values 
print len(texts)
texts = set(texts)
texts = [str(text).split() for text in texts]
print len(texts)

90432
4410


D:\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
dictionary = corpora.Dictionary(texts) # 构建词典
#print dictionary.token2id #词语转id
corpus = [dictionary.doc2bow(text) for text in texts] #每条记录是一则新闻，每则新闻有序列集(id,freq),id代表词语id,freq代表该词词频
#print corpus

tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus] #(id, tf-idf)

In [10]:
topic = 10
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=topic) 
# lsi.print_topics() 

In [11]:
lsi_vec = lsi[corpus_tfidf] # doc_num * (topic_index, topic_weight)
news_similarity_matrix = similarities.MatrixSimilarity(lsi_vec, num_features=topic)# doc_num * doc_num 

In [12]:
def get_record_by_qurey(query, data):
    title =  data[data['content'] == " ".join(query)].head(1)['title'] #可能有重复的，取第一条。把对应内容的标题返回
    return title.values[0] if len(title) > 0 else ""
    
def get_similar_news(query, topk=5, is_output_info=False):
    if is_output_info: print 'origin news:', query
    query_bow = dictionary.doc2bow(query.split())
    query_tfidf = tfidf[query_bow]
    query_tfidf_lsi = lsi[query_tfidf]
    sims = news_similarity_matrix[query_tfidf_lsi]
    sort_sims = np.array(sorted(enumerate(sims), key=lambda item: -item[1])) 
    if is_output_info:
        print 'similar news:'
        for sim in sort_sims[0:topk]:
            print get_record_by_qurey(texts[int(sim[0])], train_data), ":",sim[1]
            print
    return sort_sims[0:topk]
get_similar_news(query = "富豪榜", is_output_info=True)

origin news: 富豪榜
similar news:
亿万富翁们背后的厉害老婆 : 0.896424293518

特斯拉旋风 : 0.873941838741

十大会赚钱的明星老板 : 0.85140311718

拥有私人飞机的中国富豪 : 0.8500841856

谁的鲁能? : 0.835553228855



array([[  4.25000000e+02,   8.96424294e-01],
       [  4.07200000e+03,   8.73941839e-01],
       [  2.88900000e+03,   8.51403117e-01],
       [  1.99600000e+03,   8.50084186e-01],
       [  4.09400000e+03,   8.35553229e-01]])

In [13]:
user_no_list =  data['user_no'].unique() # 9995个用户
def get_clicked_news_by_user(user_id, data):
    return data[data['user_no'] == user_id]


def get_recommend_news_by_user(user_id):
    user_data = get_clicked_news_by_user(user_id, train_data)
    user_data_news = user_data['content']
    recommend_news = []
    for news in user_data_news:
        similar_news = get_similar_news(query=news, topk=5) # 找到和用户看过的新闻最相似的topk
        recommend_news.append(similar_news)
    recommend_news = np.concatenate(recommend_news)
    recommend_news = sorted(recommend_news, key=lambda item: item[1], reverse=True) 
    return recommend_news

def get_recommend_news_records(recommend_news):
    news_content = [" ".join(texts[int(news[0])]) for news in recommend_news]
    recommend_record = []
    for content in news_content:
        record = train_data[train_data['content'] == content].head(1)[['news_no', 'title', 'content', 'release_time']]
        recommend_record.append(record)
    
    return pd.concat(recommend_record) if len(recommend_record) > 0 else None

recommend_user_id = 4949545
recommend_news = get_recommend_news_by_user(recommend_user_id) # (编号，相似度)
recommend_record = get_recommend_news_records(recommend_news) # 对应的原始数据记录

In [14]:
recommend_record

,news_no,title,content,release_time
44663,100649193,书是什么？,这位 责备 整体 整齐 占稳 坚实 只用 一家人 疯子 讨人喜欢 爱好 发生 情愿 成分...,2014年03月10日12:46
71926,100651830,希勒：金融体系造成的不平等可以避免,金融体系 恰当 人群 诺贝尔 确保 设计 实际收入 20 当初 毕业 瑞士人 关键在于 ...,2014年03月14日21:02
65245,100651484,银监会官员：80%信托资金投向实体经济,金融机构 登记 业务 主任 非银行 体制 抗 增幅 融资 完善 20 危机 一是 近期 ...,2014年03月14日12:13
84971,100653589,劳动力成本继续构成美企在华首要挑战,显示 列美 优先 反腐败 发挥 六成 袁贵仁 合格 美企 699 矛盾 上涨 这方面 2...,2014年03月19日16:00
31344,100647806,余额宝规模增速回落流动性风险引关注,开销 常态 很难说 业务 转向 缩水 万元 途 宝 缺失 20 事情 账 热议 四大行 ...,2014年03月06日18:22
16904,100646695,调查报告：新加坡是生活最昂贵城市,显示 国家 一同 签注 调查 水资源 变得 服装 品牌服装 十座 北京 奢侈 畸高 排名...,2014年03月04日18:57
14313,100646489,李小加：H股全流通进入最后决策阶段,自然 香港媒体 过往 缓慢 值得 架构 申请 一是 转折点 总裁 港交所 2013 节奏...,2014年03月04日15:40
4681,100645720,现代版“白毛女”,那天 拎 路上 巷中 身材 1962 梳着 女人 系成 文 长得 唱戏 一根 一前一后 ...,2014年03月03日11:59
5495,100645839,数字化真的比纸张更“绿色”吗,组织 显示 国家 宜译 事 说法 电子 技术 收到 伴随 挑战 paper 只用 去除 ...,2014年03月03日15:02
6119,100645813,巴菲特2013年业绩不及标普指数,特别 一封 高于 去年 原因 增幅 ToddCombs 两位 投资 周期 复苏 高度 发...,2014年03月03日14:27
